In [1]:
# %pip install -q sentencepiece
# %pip install -q protobuf
# %pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
# %pip install -q scipy
# %pip install -q -U trl

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, PeftConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset, concatenate_datasets
import pandas as pd
import math

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from torch import nn
import re
import ast
import numpy as np

class CustomSFTTrainer(SFTTrainer):
    def __init__(self, tokenizer, *args, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False):
        inputs_copy = inputs.copy()
        labels = inputs_copy.pop("labels")
        outputs = model(**inputs_copy)
        logits = outputs.logits
        
        # Compute the loss using the superclass's compute_loss method
        super_loss, _ = super().compute_loss(model, inputs, return_outputs=True)
        print("\npreloss:", super_loss.item())
        loss = super_loss
        
        wrong_proportion = 1
        
        labels_decoded = decoded = self.tokenizer.batch_decode(labels.tolist())
        labels_result = labels_decoded[0]
        
        try:
            target_match = re.search(r"Test_1_Output=\[\[.*?\]\]", labels_result)
            target_array_string = target_match.group(0).replace("Test_1_Output=", "")
            target_array = np.array(ast.literal_eval(target_array_string))
            
            # Apply softmax to logits to get probabilities
            probs = torch.softmax(logits, dim=-1)
            # Get the predicted tokens (highest probability)
            _, predictions = torch.max(probs, dim=-1)
            
            decoded = self.tokenizer.batch_decode(predictions.tolist())
            predictions_result = decoded[0]
            match1 = re.search(r"Test_1_Output=\[\[.*?\]\]", predictions_result)
            match2 = re.search(r"Test_1_Output=\[\[.*?\],\[<\|im_end\|>", predictions_result)
            predicted_array_string = ""
            if match1:
                predicted_array_string = match1.group(0).replace("Test_1_Output=", "")
                print("matched Test_1_Output=[[*?]]")
            elif match2:
                predicted_array_string = match2.group(0).replace("Test_1_Output=", "")
                predicted_array_string = predicted_array_string.replace(",[<|im_end|>", "]")
                print("matched Test_1_Output=[[*?],[<|im_end|>")
            else:
                loss = loss * 5
                print("Does not match Test_1_Output=[[*?]] or Test_1_Output=[[*?],[<|im_end|>")
                print("label")
                print(labels_result)
                print("prediction")
                print(predictions_result)

            if predicted_array_string != "":
                try:
                    predicted_array = np.array(ast.literal_eval(predicted_array_string))
                    if predicted_array.shape == target_array.shape:
                        print("correct shape")
                        loss = loss * 0.1
                        equal_elements_count = np.sum(np.equal(target_array, predicted_array))
                        total_elements_count = target_array.size + 1
                        wrong_elements_count = total_elements_count - equal_elements_count
                        wrong_proportion = wrong_elements_count / total_elements_count
                        loss = loss * (wrong_proportion * wrong_proportion)
                        are_equal = np.array_equal(predicted_array, target_array)
                        if are_equal:
                            print("are equal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                            loss = loss * loss
                    else:
                        print("incorrect shape")
                except:
                    print("unable to create numpy array")
                    print(predicted_array_string)
                    loss = loss * 5
        except:
            pass
        
        print("wp:", wrong_proportion, "  loss:", loss.item())

        return (loss, outputs) if return_outputs else loss

In [4]:
def generate_prompt(data_point):
    user_message = data_point["instruction"]
    assistant_message = data_point["output"]
    text = f"<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{assistant_message}<|im_end|>"
    return text

In [5]:
base_model_name = "merged_models/Orca_2_finetune_04_merge"
new_adapter_name = "Orca-2-13b-4bit-finetune-on-ARC-17"

# Load the entire model on the GPU 0
device_map = {"": 0}

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [6]:
# Load the data using pandas
data_file = "data/core_ARC_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset1 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_training_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset2 = Dataset.from_pandas(df)

# Load the data using pandas
data_file = "data/ARC_augmented_evaluation_puzzles.json"
df = pd.read_json(data_file)
# Convert the pandas dataframe to a dataset
dataset3 = Dataset.from_pandas(df)

# Generate the "prompt" column for each dataset
dataset1 = dataset1.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset2 = dataset2.map(lambda data_point: {"prompt": generate_prompt(data_point)})
dataset3 = dataset3.map(lambda data_point: {"prompt": generate_prompt(data_point)})

# # Assuming dataset1 and dataset2 are instances of Dataset from the `datasets` library
dataset = concatenate_datasets([dataset1, dataset2, dataset3])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11240 [00:00<?, ? examples/s]

Map:   0%|          | 0/72504 [00:00<?, ? examples/s]

In [7]:
def main():
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)  

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, add_eos_token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

    per_device_train_batch_size = 1
    gradient_accumulation_steps = 1
    max_seq_length = 5000

    output_dir = "results/" + new_adapter_name

    steps_per_epoch = len(dataset) // (
        per_device_train_batch_size * gradient_accumulation_steps
    )
    print("Steps:", steps_per_epoch)

    # Set training parameters
    training_arguments = TrainingArguments(
        output_dir=output_dir,
        max_steps=steps_per_epoch,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim="paged_adamw_32bit",
        lr_scheduler_type="cosine",
        save_strategy="steps",
        evaluation_strategy="no",
        save_steps=1000,
        logging_steps=1,
        learning_rate=5e-6,
        fp16=True,
        warmup_steps=0.03,
        group_by_length=True,
        gradient_checkpointing=True,
    )

    trainer = CustomSFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        dataset_text_field="prompt",
        max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        args=training_arguments,
    )

    # Train model
    trainer.train()

    # Save trained model
    adapter = "adapters/" + new_adapter_name
    trainer.model.save_pretrained(adapter)

In [ ]:
main()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Steps: 93744


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/93744 [00:00<?, ? examples/s]

...


preloss: 0.1786377876996994
matched Test_1_Output=[[*?],[<|im_end|>
correct shape
wp: 0.046875   loss: 3.925146666006185e-05

preloss: 0.09489832073450089
matched Test_1_Output=[[*?]]
correct shape
wp: 0.05517241379310345   loss: 2.8887005100841634e-05

preloss: 0.09943802654743195
matched Test_1_Output=[[*?]]
correct shape
wp: 0.049586776859504134   loss: 2.445030440867413e-05

preloss: 0.22102414071559906
matched Test_1_Output=[[*?]]
incorrect shape
wp: 1   loss: 0.22102414071559906

preloss: 0.21291100978851318
matched Test_1_Output=[[*?]]
correct shape
wp: 0.12903225806451613   loss: 0.0003544824430719018

preloss: 0.22186943888664246
matched Test_1_Output=[[*?]]
correct shape
wp: 0.21621621621621623   loss: 0.0010372275719419122

preloss: 0.45677828788757324
matched Test_1_Output=[[*?]]
correct shape
wp: 0.7619047619047619   loss: 0.026515929028391838

preloss: 0.34583672881126404
matched Test_1_Output=[[*?],[<|im_end|>
correct shape
wp: 0.5   loss: 0.008645918220281601

preloss:


preloss: 0.14006994664669037
matched Test_1_Output=[[*?]]
correct shape
wp: 0.04950495049504951   loss: 3.43275060004089e-05

preloss: 0.14804379642009735
matched Test_1_Output=[[*?]]
correct shape
wp: 0.16216216216216217   loss: 0.0003893043613061309

preloss: 0.44514816999435425
matched Test_1_Output=[[*?]]
correct shape
wp: 0.17543859649122806   loss: 0.0013701082207262516

preloss: 0.25724390149116516
matched Test_1_Output=[[*?]]
correct shape
wp: 0.0625   loss: 0.00010048590047517791

preloss: 0.4990738034248352
matched Test_1_Output=[[*?]]
unable to create numpy array
[[0,0,2,0,2,0,0,2,2,0],[2,2,2,2,2],[2,2,2,2,2],[2,2,2,2,2]]
wp: 1   loss: 2.4953689575195312

preloss: 0.1673620045185089
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06097560975609756   loss: 6.222561205504462e-05

preloss: 0.3698473870754242
matched Test_1_Output=[[*?]]
correct shape
wp: 0.3793103448275862   loss: 0.005321228876709938

preloss: 0.40694454312324524
matched Test_1_Output=[[*?]]
correct shape
wp


preloss: 0.1998344212770462
matched Test_1_Output=[[*?]]
unable to create numpy array
[[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,1,1,0,0,0,0],[1,1,7,7,0,0,0,0],[1,1,7,7,0,0,0,0],[0,0,7,7,0,0,0,0],[0,0,7,7,7,7,0,0]]
wp: 1   loss: 0.9991720914840698

preloss: 0.4090690612792969
matched Test_1_Output=[[*?]]
incorrect shape
wp: 1   loss: 0.4090690612792969

preloss: 0.3250323235988617
matched Test_1_Output=[[*?]]
correct shape
wp: 0.4   loss: 0.005200516898185015

preloss: 0.3342958688735962
matched Test_1_Output=[[*?]]
correct shape
wp: 0.3   loss: 0.0030086631886661053

preloss: 0.05236786603927612
matched Test_1_Output=[[*?]]
incorrect shape
wp: 1   loss: 0.05236786603927612

preloss: 0.19503210484981537
matched Test_1_Output=[[*?]]
incorrect shape
wp: 1   loss: 0.19503210484981537

preloss: 0.2912385165691376
matched Test_1_Output=[[*?]]
correct shape
wp: 0.024896265560165973   loss: 1.8051663573714904e-05

preloss: 0.07614381611347198
matched Test_1_Output=[[*?]]
incor


preloss: 0.13198813796043396
matched Test_1_Output=[[*?]]
correct shape
wp: 0.07555555555555556   loss: 7.534730684710667e-05

preloss: 0.08890384435653687
matched Test_1_Output=[[*?]]
correct shape
wp: 0.1951219512195122   loss: 0.00033847984741441905

preloss: 0.15624286234378815
matched Test_1_Output=[[*?],[<|im_end|>
unable to create numpy array
[[3,3,8,3],[0,3],[8,3]]
wp: 1   loss: 0.7812142968177795

preloss: 0.5959562659263611
matched Test_1_Output=[[*?],[<|im_end|>
correct shape
wp: 0.025423728813559324   loss: 3.8520585803780705e-05

preloss: 0.06437834352254868
matched Test_1_Output=[[*?]]
correct shape
wp: 0.05309734513274336   loss: 1.8150367395719513e-05

preloss: 0.04353863745927811
matched Test_1_Output=[[*?]]
correct shape
wp: 0.015432098765432098   loss: 1.0368712537456304e-06

preloss: 0.09385590255260468
matched Test_1_Output=[[*?]]
unable to create numpy array
[[5.0,6.0,5.0,5.0],[4.0,4.0],[6.0,6.0,5.0,5.0,4.0,4.0],[4.0,4.0,7.0,4.0,7.0,7.0],[4.0,4.0,4.0,4.0,7.0,7.0]


preloss: 1.1209889650344849
matched Test_1_Output=[[*?]]
correct shape
wp: 0.6153846153846154   loss: 0.0424516536295414

preloss: 0.26300206780433655
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06930693069306931   loss: 0.00012633175356313586

preloss: 0.07987187057733536
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06369426751592357   loss: 3.240369915147312e-05

preloss: 0.16228261590003967
matched Test_1_Output=[[*?]]
incorrect shape
wp: 1   loss: 0.16228261590003967

preloss: 0.30693113803863525
matched Test_1_Output=[[*?]]
correct shape
wp: 0.11764705882352941   loss: 0.00042481819400563836

preloss: 0.10532109439373016
matched Test_1_Output=[[*?]]
correct shape
wp: 0.034482758620689655   loss: 1.2523317309387494e-05

preloss: 0.3709256649017334
matched Test_1_Output=[[*?]]
correct shape
wp: 0.5294117647058824   loss: 0.010396186262369156

preloss: 0.1126275286078453
matched Test_1_Output=[[*?]]
correct shape
wp: 0.06097560975609756   loss: 4.187519880360924e-05

prelos